In [1]:
import sys
sys.path.insert(0, '../..')
import os.path
import copy
import numpy as np
from model.base import CovidModel, SimulationParameters, set_parameters, normal_ci
from utils import BasicStatistics, RemovePolicy, Propaganda, Vaccination, setup_city_layout, AddPolicyInfectedRate, confidence_interval, multiple_runs
from model.utils import SocialPolicy, WorkClasses

In [2]:
common_parameters = SimulationParameters(
    latency_period_shape = 3,
    latency_period_scale = 1, 
    incubation_period_shape = 6,
    incubation_period_scale = 1, 
    mild_period_duration_shape = 14,
    mild_period_duration_scale = 1,
    hospitalization_period_duration_shape = 12,
    hospitalization_period_duration_scale = 1,
    spreading_rate = normal_ci(2.41, 3.90, 20),
    initial_infection_rate = 0.01,
    imune_rate = 0.01,
    hospitalization_capacity = 0.05,
    icu_capacity = 0.03,
    symptomatic_isolation_rate = 0.0,
    asymptomatic_contagion_probability = 0.1,
    allowed_restaurant_capacity = 0.50, # valid values: {1.0, 0.50, 0.25}
    mask_user_rate = 0.9, # 0.0,
    mask_efficacy = 0.6, #0.0,
    risk_tolerance_mean = 0.7,
    risk_tolerance_stdev = 0.2,
    herding_behavior_mean = 0.7,
    herding_behavior_stdev = 0.2,
    restaurant_count_per_work_district = 10, #100
    hoprank_infected_sample_ratio = 1.0
)

population_size = 1000
simulation_cycles = 450 # days
seed = 31415
np.random.seed(seed)

params = copy.deepcopy(common_parameters)

In [3]:
available_number_of_vaccine_shots = population_size
start_month = [10]
monthly_vaccination_capacity = [0.10]
vaccination_scenarios = [
    ("A1", [
        (30, [WorkClasses.HOSPITAL], 80, 1),
        (30, [WorkClasses.HOSPITAL, WorkClasses.TEACHER], 60, 1),
        (30, [WorkClasses.HOSPITAL, WorkClasses.TEACHER], 60, 2),
        (30, None, 0, 1)
    ]),
    ("A2", [
        (30, [WorkClasses.HOSPITAL], 80, 1),
        (30, [WorkClasses.HOSPITAL, WorkClasses.TEACHER], 60, 1),
        (60, None, 0, 1)
    ]),
    ("B1", [
        (30, [WorkClasses.HOSPITAL, WorkClasses.TEACHER], None, 1),
        (30, None, 60, 1),
        (60, None, 0, 1)
    ]),
    ("B2", [
        (30, [WorkClasses.HOSPITAL, WorkClasses.TEACHER], None, 1),
        (30, [WorkClasses.HOSPITAL, WorkClasses.TEACHER], None, 2),
        (30, None, 60, 1),
        (30, None, 0, 1)
    ]),
    ("C1", [
        (120, None, 0, 1)
    ]),
    ("C2", [
        (60, None, 0, 1),
        (60, None, 0, 2)
    ])
]

interval_between_shots = 21
vaccination_listeners = []
for m in start_month:
    for r in monthly_vaccination_capacity:
        start = 30 * (m - 1) + 1
        for scenario, stages in vaccination_scenarios:
            if scenario != "B2":
                vaccination_listeners.append((f"{scenario}_{m}_{r}_lockdown1", [
                    ["Vaccination", start, r * population_size, population_size, interval_between_shots, stages],
                    ["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_ELEMENTARY_SCHOOL,0.025 ,1],
                    ["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_MIDDLE_SCHOOL, 0.025 ,1],
                    ["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_HIGH_SCHOOL, 0.025 ,1],
                    ["RemovePolicy", SocialPolicy.LOCKDOWN_ELEMENTARY_SCHOOL, start + 30],
                    ["RemovePolicy", SocialPolicy.LOCKDOWN_MIDDLE_SCHOOL, start + 30],
                    ["RemovePolicy", SocialPolicy.LOCKDOWN_HIGH_SCHOOL, start + 30]
                ]))
            vaccination_listeners.append((f"{scenario}_{m}_{r}_lockdown2", [
                ["Vaccination", start, r * population_size, population_size, interval_between_shots, stages],
                ["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_ELEMENTARY_SCHOOL,0.025 ,1],
                ["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_MIDDLE_SCHOOL, 0.025 ,1],
                ["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_HIGH_SCHOOL, 0.025 ,1],
                ["RemovePolicy", SocialPolicy.LOCKDOWN_ELEMENTARY_SCHOOL, start + 60],
                ["RemovePolicy", SocialPolicy.LOCKDOWN_MIDDLE_SCHOOL, start + 60],
                ["RemovePolicy", SocialPolicy.LOCKDOWN_HIGH_SCHOOL, start + 60]
            ]))
            vaccination_listeners.append((f"{scenario}_{m}_{r}_lockdownfull", [
                ["Vaccination", start, r * population_size, population_size, interval_between_shots, stages],
                ["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_ELEMENTARY_SCHOOL,0.025 ,1],
                ["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_MIDDLE_SCHOOL, 0.025 ,1],
                ["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_HIGH_SCHOOL, 0.025 ,1],
                ["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_RETAIL,0.025 ,1],
                ["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_FACTORY, 0.025 ,1],
                ["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_OFFICE, 0.025 ,1],
                ["RemovePolicy", SocialPolicy.LOCKDOWN_ELEMENTARY_SCHOOL, start + 30],
                ["RemovePolicy", SocialPolicy.LOCKDOWN_MIDDLE_SCHOOL, start + 30],
                ["RemovePolicy", SocialPolicy.LOCKDOWN_HIGH_SCHOOL, start + 30],
                ["RemovePolicy", SocialPolicy.LOCKDOWN_RETAIL, start + 30],
                ["RemovePolicy", SocialPolicy.LOCKDOWN_FACTORY, start + 30],
                ["RemovePolicy", SocialPolicy.LOCKDOWN_OFFICE, start + 30]
            ]))

In [4]:
school_home_list = [
[
(0,0),(0,1),(0,2),(0,3),(0,4),
(0,5),(0,6),(0,7),(0,8),(0,9),
(1,0),(1,1),(1,2),(1,3),(1,4),
(1,5),(1,6),(1,7),(1,8),(1,9)],
[
(2,0),(2,1),(2,2),(2,3),(2,4),
(2,5),(2,6),(2,7),(2,8),(2,9),
(3,0),(3,1),(3,2),(3,3),(3,4),
(3,5),(3,6),(3,7),(3,8),(3,9)],
[
(4,0),(4,1),(4,2),(4,3),(4,4),
(4,5),(4,6),(4,7),(4,8),(4,9),
(5,0),(5,1),(5,2),(5,3),(5,4),
(5,5),(5,6),(5,7),(5,8),(5,9)],
[
(6,0),(6,1),(6,2),(6,3),(6,4),
(6,5),(6,6),(6,7),(6,8),(6,9),
(7,0),(7,1),(7,2),(7,3),(7,4),
(7,5),(7,6),(7,7),(7,8),(7,9)]
]

work_home_list = [
[
(0,0),(0,1),(0,2),(0,3),(0,4),
(0,5),(0,6),(0,7),(0,8),(0,9),
(1,0),(1,1),(1,2),(1,3),(1,4),
(1,5),(1,6),(1,7),(1,8),(1,9),

(2,0),(2,1),(2,2),(2,3),(2,4),
(2,5),(2,6),(2,7),(2,8),(2,9),
(3,0),(3,1),(3,2),(3,3),(3,4),
(3,5),(3,6),(3,7),(3,8),(3,9),


(4,0),(4,1),(4,2),(4,3),(4,4),
(4,5),(4,6),(4,7),(4,8),(4,9),
(5,0),(5,1),(5,2),(5,3),(5,4),
(5,5),(5,6),(5,7),(5,8),(5,9),

(6,0),(6,1),(6,2),(6,3),(6,4),
(6,5),(6,6),(6,7),(6,8),(6,9),
(7,0),(7,1),(7,2),(7,3),(7,4),
(7,5),(7,6),(7,7),(7,8),(7,9)
]]

In [5]:
%%capture cap --no-stderr

#for temperature in [-1, -0.998, -0.9978, -0.9975]:

for temperature in [-1, -0.998]:
    for risk_tolerance_mean in [0.1, 0.7]:
        for herding_behavior_mean in [0.1, 0.7]:
            for scenario, listeners in vaccination_listeners:
                params = copy.deepcopy(common_parameters)
                params.params['vaccine_immunization_rate'] = [0.4, 0.8]
                params.params['vaccine_symptom_attentuation'] = [0.4, 0.8]
                params.params['num_communities'] = 20
                params.params['temperature'] = temperature
                params.params['risk_tolerance_mean'] = risk_tolerance_mean
                params.params['herding_behavior_mean'] = herding_behavior_mean
                print("================================================================================")
                print(f"temperature = {temperature}")
                print(f"risk_tolerance_mean = {risk_tolerance_mean}")
                print(f"herding_behavior_mean = {herding_behavior_mean}")
                print(f"scenario = {scenario}")
                print("================================================================================")
                output_filename = f"vaccination_{population_size}_{scenario}_{temperature}_{risk_tolerance_mean}_{herding_behavior_mean}"
                if os.path.exists(output_filename + '.png'):
                    continue
                multiple_runs(
                    params,
                    population_size,
                    simulation_cycles,
                    num_runs=10,
                    fname=f"vaccination_{population_size}_{scenario}_{temperature}_{risk_tolerance_mean}_{herding_behavior_mean}", 
                    do_print=True, 
                    home_grid_height=8, 
                    home_grid_width=10, 
                    school_home_list=school_home_list,
                    work_home_list=work_home_list,
                    compute_hoprank=False,
                    listeners=listeners)


KeyboardInterrupt: 

In [6]:
with open('notebook_log2.txt', 'w') as f:
    f.write(cap.stdout)